In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

Cloning into 'yolov5'...
remote: Enumerating objects: 14906, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 14906 (delta 11), reused 13 (delta 2), pack-reused 14880
Receiving objects: 100% (14906/14906), 13.87 MiB | 33.11 MiB/s, done.
Resolving deltas: 100% (10256/10256), done.
/content/yolov5
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


In [3]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 1.6 MB 5.1 MB/s 
Setup complete. Using torch 1.13.0+cu116 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [4]:
#follow the link below to get your download code from from Roboflow
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="Far5TQ0Gqf6CkU9fHs95")
project = rf.workspace("uav-project-eia1l").project("uav_ir")
dataset = project.version(6).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 825 kB/s 
     |████████████████████████████████| 178 kB 8.8 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 
     |████████████████████████████████| 145 kB 45.1 MB/s 
     |████████████████████████████████| 138 kB 74.7 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=8a0db52aa93e5ee65909dae470e2c991753cf2baac31fd5e6062a607bc826a25
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: certifi
    Found existing installation: certif

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to UAV_IR-6 in yolov5pytorch:: 100%|██████████| 7847/7847 [00:02<00:00, 3514.31it/s]


In [5]:
%cd /content/yolov5
%cat {dataset.location}/data.yaml

/content/yolov5
names:
- '0'
nc: 1
train: UAV_IR-6/train/images
val: UAV_IR-6/valid/images


In [6]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [7]:
#this is the model configuration we will use for our tutorial 
%cat //content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [8]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [32]:
%%writetemplate /content/yolov5/models/TFNet.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors: 3  # AutoAnchor evolves 3 anchors per P output layer
backbone: 
  [[-1, 1, Conv, [64, 3, 2]],  # 0-P1/2
   [-1, 3, BottleneckCSP, [64]], #1
   [-1, 1, Conv, [128, 5, 2]],  # 2-P2/4
   [-1, 3, BottleneckCSP, [128]], #3
   [-1, 1, Conv, [256, 7, 2]],  # 4-P3/8
   [-1, 9, BottleneckCSP, [256]], #5
   [-1, 1, SPPF, [256, 5]],  # 1
  ]
# YOLOv5 head
head:
  [[-1, 1, Conv, [64, 3, 2]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 1], 1, Concat, [1]],  # cat backbone P1
   [-1, 3, BottleneckCSP, [64, False]],  # 10

   [-1, 1, Conv, [128, 3, 2]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 3], 1, Concat, [1]],  # cat backbone P2
   [-1, 3, BottleneckCSP, [128, False]],  # 14 (P2/medium)

   [-1, 1, Conv, [256, 5, 2]],
   [[-1, 5], 1, Concat, [1]],  # cat head P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/16-medium)

   [[10, 14, 17], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [33]:
# train model for first time
%%time
%cd /content/yolov5/
!python train.py --img 320 --batch-size -1 --epochs 1000 --optimizer Adam --data {dataset.location}/data.yaml --cfg ./models/TFNet.yaml --hyp data/hyps/hyp.scratch-high.yaml --weights '' --project "/content/gdrive/MyDrive/Runs"  --name TFNet  --cache

/content/yolov5
train: weights=, cfg=./models/TFNet.yaml, data=/content/yolov5/UAV_IR-6/data.yaml, hyp=data/hyps/hyp.scratch-high.yaml, epochs=1000, batch_size=-1, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=Adam, sync_bn=False, workers=8, project=/content/gdrive/MyDrive/Runs, name=TFNet, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 417 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_

In [ ]:
#Later, if it is stopped prematurely and I need to resume, I use
#!python train.py --img 320 --weights '' --project "/content/gdrive/MyDrive/Runs"  --name TFNet  --cache  --resume "/content/gdrive/MyDrive/Runs/TFNet/weights/last.pt"

In [ ]:
%cd /content/yolov5/
!python detect.py --weights /content/gdrive/MyDrive/Runs/TFNet/weights/best.pt --img 320 --conf 0.3 --source /content/yolov5/Uav-11/valid/images 

In [ ]:
!python val.py --task study --weights /content/gdrive/MyDrive/Runs/TFNet/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5

In [ ]:
!python val.py --task speed --weights /content/gdrive/MyDrive/Runs/TFNet/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5

In [ ]:
!python val.py --img 320 --weights /content/gdrive/MyDrive/Runs/TFNet/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5 

val: data=/content/yolov5/Uav-11/data.yaml, weights=['/content/gdrive/MyDrive/Runs/MFNet_small2_results/weights/best.pt'], batch_size=32, imgsz=320, conf_thres=0.001, iou_thres=0.5, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 232 layers, 1615007 parameters, 0 gradients, 17.3 GFLOPs
val: Scanning '/content/yolov5/Uav-11/valid/labels.cache' images and labels... 365 found, 0 missing, 0 empty, 0 corrupt: 100% 365/365 [00:00<?, ?it/s]
               Class     Images     Labels         TP         FP          P          R         F1     mAP@.5 mAP@.5:.95: 100% 12/12 [00:04<00:00,  2.98it/s]
                 all        365        365        174          9      0.942      0.951      0.947      0.963       0.6

In [ ]:
#%cd /content/yolov5/
#!python detect.py --weights runs/train/MFNet_small2_results/weights/best.pt --img 320 --conf 0.3 --source /content/yolov5/Uav-7/valid/images 

In [ ]:
#### Visulize each layers feature map
#######Run this cell to know the whats inside every lare feature
#### stop it ur self after one image is done
## Print whole folder feature maps and save them
#!python detect.py --weights /content/yolov5/mfnets.pt --img 416 --conf 0.4 --source /content/yolov5/Uav-7/test/images --visualize
####### Only single image
#!python detect.py --weights /content/yolov5/mfnets.pt --img 416 --conf 0.4 --source /content/yolov5/Uav-7/test/images/0094_jpg.rf.09856e96b7f9f77c2bc502452ec20d77.jpg --visualize
                                               

In [ ]:
#!python val.py --task study --weights runs/train/MFNet_small2_results2/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5

In [ ]:
#!python val.py --task speed --weights runs/train/MFNet_small2_results2/weights/best.pt--data {dataset.location}/data.yaml --iou 0.5

In [ ]:
#!python val.py --img 320 --weights runs/train/MFNet_small2_results2/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5 

In [ ]:
!zip -r /content/TFNet-nightvision-Adam.zip /content/gdrive/MyDrive/Runs/
!zip -r /content/TFNet-nightvision-Adam.zip /content/yolov5/runs/
from google.colab import files
files.download("/content/TFNet-nightvision-Adam.zip")